<a href="https://colab.research.google.com/github/edurso/obj-detect/blob/master/notebooks/obj_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Object Detection Model Training Notebook

structure of project:

```
training_demo/
├─ annotations/
│  └─ label_map.pbtxt
├─ exported_models/
├─ images/
│  ├─ test/
│  └─ train/
├─ models/
│  └─ ${MODEL}/
│     └─ pipeline.config
└─ pretrained_models/
   └─ ${MODEL}/
```

where `${MODEL}` is a model from the [object detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)

mount data from google drive

In [ ]:
workdir = '/content/workspace/MyDrive/tensorflow/rapid_react/'
mname = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8' # from model zoo
mlink = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/{}.tar.gz'.format(mname) # link to model zoo
from google.colab import drive
import os
!mkdir -p /content/workspace
drive.mount('/content/workspace')

## Set Up Workspace

make `workdir` if it doesn't exist

In [ ]:
import pathlib

os.chdir('/')
if not pathlib.Path(workdir).exists():
  !mkdir -p {workdir}

create working directory tree & download and extract model

In [ ]:
# make directory structure
os.chdir(workdir)
if not pathlib.Path(workdir + 'models/').exists():
  !mkdir -p annotations/ exported_models/ pretrained_models/ images/

  # download pretrained model
  !wget {mlink}
  !tar -xvzf {mname}.tar.gz
  !mv {mname}/ pretrained_models/{mname}
  !rm -rf {mname}.tar.gz

  # make directory for model
  !mkdir -p {workdir}models/{mname}
  # !cp -r {workdir}pretrained_models/{mname}/checkpoint {workdir}models/{mname}/
  !cp {workdir}pretrained_models/{mname}/pipeline.config {workdir}models/{mname}/

*Items now need to be uploaded/modified:*

- `label_map.pbtxt` to `annotations/`
- training and testing images/labels to `images/` after creating it with [this script](https://github.com/edurso/obj-detect/blob/master/scripts/partition-dataset.py)
- `models/pipeline.config` needs to be modified to reflect the dataset

*Pipeline Configuration:*

- `model.ssd.num_classes`: set this to the number of different label classes as defined in `annotations/label-map.pbtxt`
- `train_config.batch_size`: set this to the desired batch size (larger batch sizes require more memory during training)
- `train_input_reader.label_map_path: "annotations/label_map.pbtxt"`
- `train_input_reader.tf_record_input_reader.input_path: "annotations/train.record"`
- `eval_config.metrics_set: "coco_detection_metrics"`: optional
- `eval_config.use_moving_averages: false`: optional
- `eval_input_reader.label_map_path: "annotations/label_map.pbtxt"`
- `eval_input_reader.tf_record_input_reader.input_path: "annotations/test.record"`

Depending on the model, additional configuration may be required.

In [ ]:
assert pathlib.Path(workdir + 'annotations/label_map.pbtxt').is_file()

## TensorFlow Object Detection API Installation

clone tf models repo & utility repo (if not already present)

In [ ]:
import pathlib

os.chdir('/content/')

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models /content/models

if "utils" in pathlib.Path.cwd().parts:
  while "utils" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('utils').exists():
  !git clone --depth 1 https://github.com/edurso/obj-detect /content/utils

install object_detection package

In [ ]:
%%bash

# update pip
python3 -m pip install --upgrade pip

# check opencv dependency versions
python3 -m pip install chainer
python3 -m pip uninstall opencv-python-headless==4.5.5.62 
python3 -m pip install opencv-python-headless==4.1.2.30

# compile protos and install object_detection
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python3 -m pip install . --quiet

test installation

In [ ]:
!python3 /content/models/research/object_detection/builders/model_builder_tf2_test.py

check version

In [ ]:
!pip3 show object-detection

update cudnn

In [ ]:
!dpkg --configure -a
!apt-get install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!apt-get install --allow-change-held-packages libcudnn8-dev=8.1.0.77-1+cuda11.2

In [ ]:
!nvcc --version
!nvidia-smi
import chainer
chainer.print_runtime_info()

check tensorflow version

In [ ]:
import tensorflow as tf
print(tf.__version__)

## Data Pre-Processing

create tfrecords

In [ ]:
from time import time
os.chdir(workdir)
start = time()

# Generate TFRecord for Training Data
!python3 /content/utils/scripts/generate-tfrecord.py \
    -x images/train \
    -l annotations/label_map.pbtxt \
    -o annotations/train.record

# Generate TFRecord for Validation Data
!python3 /content/utils/scripts/generate-tfrecord.py \
    -x images/test \
    -l annotations/label_map.pbtxt \
    -o annotations/test.record

print('records generated in {}s'.format(time()-start))

## Training

start tensroboard monitoring

In [ ]:
os.chdir(workdir)
%load_ext tensorboard
%tensorboard --logdir=models/{mname}

start training job

In [ ]:
os.chdir(workdir)
!python3 /content/models/research/object_detection/model_main_tf2.py \
    --include masks \
    --model_dir=models/{mname} \
    --pipeline_config_path=models/{mname}/pipeline.config

## Export Trained Model

export model

In [ ]:
os.chdir(workdir)
!mkdir -p {workdir}exported_models/trained_model/
!python3 /content/models/research/object_detection/exporter_main_v2.py \
    --input_type float_image_tensor \
    --pipeline_config_path ./models/{mname}/pipeline.config \
    --trained_checkpoint_dir ./models/{mname}/ \
    --output_directory {workdir}exported_models/trained_model/

copy label file to deployable model

In [ ]:
!cp {workdir}annotations/label_map.pbtxt {workdir}exported_models/trained_model/label_map.pbtxt
!cat {workdir}annotations/label_map.pbtxt

the [`tensorrt.ipynb`](https://github.com/edurso/obj-detect/blob/master/notebooks/tensorrt.ipynb) notebook will use this to build a tensorrt engine